# Automating excel files to databases

### Overview:

Using some test data obtained from the International Coffee Organization (ICO), I have created two workflows which transform these excel reports into the third normal form of data used for databases. This jupyter notebook will feature the method using python.

The end result is a five seperate csv files for the time series data and one file for the country attributes. Depending on how this data were to be consumed, the data could be further refined for an accounting journal entry, forecast, analyzed in a visualization software, or be dashboarded on the cloud using an approach susch as AWS using S3 -> Glue-> Athena-> Tableau. 

In four of the five data sets taken from the ICO, the sum of the country values did not foot to their totals shown in the spreadsheet. I verified this by opening the excel files and manually summing the totals for each country as well as searching for hidden rows. I can only speculate as to why their totals did not sum properly since their totals were hard coded in the excel files and no explanation was provided. However, all the errors were between 1998 and 2012 and was likely an excel formula error and not intentional. 

Data can be found at:
http://www.ico.org/new_historical.asp

### Background:

In my previous role as a financial analyst, my days were often spent executing a series of transformatative steps within an excel spreadsheet until I had a final product in the form of a report/dashboard, updated analysis, or source data for an accounting journal entry. These non-adhoc tasks have well documented steps, consistent source data, and are performed on a fixed schedule. Tasks like these are ripe for automation. Any solution will take the form of either a coded, no-code, or a blended approach with each having their pros and cons. Writing a scripted code in a language such as python offers more flexibility, however, it has a higher learning curve. No-code, propietary solutions such as Tableau Prep or Alteryx offer a more user friendly interface but less flexibility and a higher price tag. A no-code software solution also has the benefit of being less reliant on any one individual for their unique skills. In the event of attrition, another team member could more easily inheret the responsibilities of a no-code or blended workflow compared to a coded solution run outside of tableau (like a jupyter notebook). A blended solution is ideal and I beleive it to be the future, however, I have found Pytab (for Tableau) to have an overly complicated integration into the tableau environment. 

In [48]:
import pandas as pd
import io
import requests
import re
import numpy as np
url="http://www.ico.org/historical/1990%20onwards/Excel/1a%20-%20Total%20production.xlsx"
#s=requests.get(url).content
c=pd.read_excel(url, header=3) #Read in data to pandas and eliminate columns before the true headers 

In [49]:
c.columns = ['Country', 'Beans']+[x for x in c.columns for x in re.findall(r'^\d{4}', x)] #Clean headers 

In [50]:
c = c.loc[c.iloc[:,2].isna()==False] #eliminate empty rows (white space)

In [51]:
check_digit = c.loc[c.Country.str.contains('total', flags=re.I,regex=True),'1990':].sum() #will be used to tie out once finished

In [52]:
c = c.loc[~c.Country.str.contains('total', flags=re.I,regex=True),:] #eliminate totals columns

In [53]:
c.insert(1, 'Harvest_Month', np.nan) #insert empty column to serve as destination for harvest_month

Here's how the data looks so far...

In [54]:
c.head()

,Country,Harvest_Month,Beans,1990,1991,1992,1993,1994,1995,1996,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
1,April group,NaN,NaN,40795.2926,42532.5454,46870.7042,40176.7776,39995.291,29402.0492,44301.040,...,61582.4778,69823.8707,66406.2131,75907.6048,74428.1052,68789.9091,70857.4583,75456.8828,68598.4217,78360.0000
2,Angola,NaN,(R/A),50.3450,79.3310,77.5200,32.6080,76.802,62.1090,70.925,...,13.4200,34.9700,28.7150,32.7900,34.9350,39.4050,40.5150,44.8300,35.0060,40.3874
3,Bolivia (Plurinational State of),NaN,(A),122.7770,103.5360,120.2350,50.8230,116.944,142.4850,124.579,...,128.4751,117.2249,131.8354,105.2812,119.9122,99.8766,84.2191,77.9835,83.8112,82.5687
4,Brazil,NaN,(A/R),27285.6286,27293.4934,34603.3542,28166.9786,28192.047,18060.2022,29196.743,...,43976.8120,55428.4102,48591.8289,55418.0012,54688.9664,53304.7669,52870.5876,56788.1784,52739.8635,62924.8836
5,Burundi,NaN,(A/R),487.3930,667.1990,620.2380,393.3540,664.143,433.9800,400.969,...,111.6130,352.9776,204.1328,405.9615,163.2177,247.5500,274.1017,248.7933,202.1079,178.4206


In [55]:
x = c[c.Beans.isna()==True].index #get indexes of subheaders
c.loc[x,'Harvest_Month'] = c.Country.loc[c.Beans.isna()==True] #move subheaders to new column 'Harvest_Month'
c.Harvest_Month = c.Harvest_Month.fillna(method='ffill') #forward fill harvest month assignments
c.Harvest_Month = [x.split(' ')[0] for x in c.Harvest_Month] #remove the word 'group' (optional step)
c = c.loc[~c.Country.str.contains('group', flags=re.I,regex=True),:] #drop total rows containing subheader

As described previously, the provided totals in the source data do not match the source detail data. This an error on behalf of the ICO and not the script. The variances are shown below.

In [56]:
c.iloc[:,3:].sum()-check_digit

1990       0.0
1991       0.0
1992       0.0
1993       0.0
1994       0.0
1995       0.0
1996       0.0
1997       0.0
1998    -163.0
1999    -151.0
2000    -197.0
2001    -163.0
2002   -1113.0
2003   -1386.0
2004   -1146.0
2005    -622.0
2006    -614.0
2007    -621.0
2008    -620.0
2009    -610.0
2010    -450.0
2011    -500.0
2012   -1250.0
2013       0.0
2014       0.0
2015       0.0
2016       0.0
2017       0.0
2018       0.0
dtype: float64

In [57]:
c.iloc[:,3:].sum().sum()-3588429.34 #Manual sum of values in in excel to verify discrepency is not a result of the script

0.0010000006295740604

In [58]:
#pivot years data to put data in first normal form
c = c.set_index(list(c.iloc[:,:3].columns)).stack().reset_index().rename(columns={'level_3': 'Year', 0:'Value'}) #pivot years data to normalize

Here I've pivoted the data to make the year a column instead of a seperate header

In [59]:
c.head()

,Country,Harvest_Month,Beans,Year,Value
0,Angola,April,(R/A),1990,50.345
1,Angola,April,(R/A),1991,79.331
2,Angola,April,(R/A),1992,77.520
3,Angola,April,(R/A),1993,32.608
4,Angola,April,(R/A),1994,76.802


Here we split off to be Harvest month and beans to be normalization purists and save some cloud storage costs by eliminating data repitition. This step is optional depending on how the data is to be consumed. Below the final table

In [60]:
d = c.iloc[:,:3] #drop year and value cols and create a country attribute table
d = d.drop_duplicates(keep = 'first') #reduce to unique combinations
c = c.drop(['Harvest_Month', 'Beans'], axis = 1) #drop country attribute variables to make purely time series table
c

,Country,Year,Value
0,Angola,1990,50.3450
1,Angola,1991,79.3310
2,Angola,1992,77.5200
3,Angola,1993,32.6080
4,Angola,1994,76.8020
...,...,...,...
1619,Yemen,2014,178.3814
1620,Yemen,2015,156.2460
1621,Yemen,2016,143.4533
1622,Yemen,2017,132.1281


# Automating the rest of the files more concisely

In the approach below, I have pasted urls to the files. If the files were in a target folder, we could have easily looped through the folder contents to pull in all the files in that folder as well. Ultimately, I condensed the previous steps and placed them within a for loop to apply the procedures to each file. The code also saves each table into a dict as well as a csv file on my local drive. We could have uploaded the these files to an AWS S3 bucket as well within the loop using Boto3. 

In [61]:
Total_production = 'http://www.ico.org/historical/1990%20onwards/Excel/1a%20-%20Total%20production.xlsx'
domestic_consumption = 'http://www.ico.org/historical/1990%20onwards/Excel/1b%20-%20Domestic%20consumption.xlsx'
Exportable_production = 'http://www.ico.org/historical/1990%20onwards/Excel/1c%20-%20Exportable%20production.xlsx'
Gross_opening_stocks = 'http://www.ico.org/historical/1990%20onwards/Excel/1d%20-%20Gross%20Opening%20stocks.xlsx'
Exports = 'http://www.ico.org/historical/1990%20onwards/Excel/1e%20-%20Exports%20-%20crop%20year.xlsx'
filelist = [Total_production, domestic_consumption, Exportable_production, Gross_opening_stocks, Exports]
filenames = ['Total_production', 'domestic_consumption', 'Exportable_production', 'Gross_opening_stocks', 'Exports']

In [62]:
path = 'C:/Users/erler/OneDrive/Documents/Random Data Sets/coffee/Finished/Python_method/'
tables = {}
for j,g in enumerate(filelist):    
    c=pd.read_excel(g, header=3) #Read in data to pandas and eliminate columns before the true headers
    c.columns = ['Country', 'Beans']+[x for x in c.columns for x in re.findall(r'^\d{4}', x)] #Clean headers 
    c = c.loc[c.iloc[:,2].isna()==False] #eliminate empty rows (white space)
    check_digit = c.loc[c.Country.str.contains('total', flags=re.I,regex=True),'1990':].sum() #used to tie out once finished
    c = c.loc[~c.Country.str.contains('total', flags=re.I,regex=True),:] #eliminate totals columns
    c.insert(1, 'Harvest_Month', np.nan) #Insert empty column to serve as destination for harvest_month
    x = c[c.Beans.isna()==True].index #Get indexes of subheaders
    c.loc[x,'Harvest_Month'] = c.Country.loc[c.Beans.isna()==True] #move subheaders to new column 'Harvest_Month'
    c.Harvest_Month = c.Harvest_Month.fillna(method='ffill') #forward fill harvest month assignments
    c.Harvest_Month = [x.split(' ')[0] for x in c.Harvest_Month] #remove the word 'group' (optional step)
    c = c.loc[~c.Country.str.contains('group', flags=re.I,regex=True),:] #drop total rows containing subheader
    c = c.set_index(list(c.iloc[:,:3].columns)).stack().reset_index().rename(columns={'level_3': 'Year', 0:'Value'}) #pivot years data to normalize
    d = c.iloc[:,:3] #drop year and value cols and create a country attribute table
    d = d.drop_duplicates(keep = 'first') #reduce to unique combinations
    c = c.drop(['Harvest_Month', 'Beans'], axis = 1) #drop country attribute variables to make purely time series table
    print(filenames[j]+' variance:', c.Value.sum()-check_digit.sum()) #check for variance
    c.to_csv(path+str(filenames[j])+'.csv') #save to folder 
    tables[filenames[j]] = c #make dict of all tables

Total_production variance: -9606.000000000466
domestic_consumption variance: -4809.0
Exportable_production variance: -4991.0
Gross_opening_stocks variance: 0.0
Exports variance: -113435.0064000003


In [63]:
tables['Total_production'] #dict of tables

,Country,Year,Value
0,Angola,1990,50.3450
1,Angola,1991,79.3310
2,Angola,1992,77.5200
3,Angola,1993,32.6080
4,Angola,1994,76.8020
...,...,...,...
1619,Yemen,2014,178.3814
1620,Yemen,2015,156.2460
1621,Yemen,2016,143.4533
1622,Yemen,2017,132.1281
